In [108]:
%load_ext autoreload
%autoreload 2

import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import *
import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, os.environ["PROJECT_DIR"] + "/estimations/estimation-current")
os.chdir(os.environ["PROJECT_DIR"] + "/estimations/estimation-current")
from moments import get_moments

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
fname = os.environ["PROJECT_DIR"] + "/resources/soepcore_struct_prep.dta"
df_obs = pd.read_stata(fname, convert_categoricals=False)
df_obs = df_alignment(df_obs)

params_start = pd.read_pickle("start.soepy.pkl")
fname  = "resources/model_spec_init.yml"
df_sim = df_alignment(soepy.simulate(params_start, fname))

In [106]:
_ = get_moments(df_obs)

In [107]:
df_grid = pd.read_pickle("df-grid.pkl")
df_info = pd.read_pickle("df-info.pkl")


df_grid

Value
Period Choice           
0      Part    21.364945
       Full    13.130414
1      Part    21.042286
       Full    15.827183
2      Part    19.573086
       Full    15.555245
3      Part    27.624391
       Full    18.117759
4      Part    27.266172
       Full    19.868371
5      Part    27.574928
       Full    24.819533
6      Part    38.195875
       Full    30.674677
7      Part    39.697779
       Full    29.808751
8      Part    36.282993
       Full    42.053535
9      Part    41.637965
       Full    46.256003
10     Part    33.874744
       Full    51.496356
11     Part    39.171488
       Full    54.762800
12     Part    42.167559
       Full    52.524845
13     Part    45.342921
       Full    50.249545
14     Part    52.602359
       Full    60.750801
15     Part    53.711830
       Full    57.642062
16     Part    61.544932
       Full    61.705360
17     Part    59.873097
       Full    68.954989
18     Part    60.604993
       Full    64.371884
19     Part    59.051963
       Full    72.251503
20     Part    63.216499
       Full    72.539745
21     Part    61.727981
       Full    68.298139
22     Part    62.817816
       Full    68.592149
23     Part    63.616450
       Full    58.585269
24     Part    51.979538
       Full    63.797498
25     Part    51.393217
       Full    68.210911
26     Part    55.588691
       Full    69.450247
27     Part    48.085099
       Full    75.444682
28     Part    59.832384
       Full    69.816664
29     Part    55.945102
       Full    70.557220
30     Part    57.726648
       Full    72.607834
31     Part    54.876122
       Full    76.719382
32     Part    55.609850
       Full    69.272264
33     Part    57.363777
       Full    72.490263
34     Part    52.715535
       Full    72.073542
35     Part    49.949654
       Full    73.645219
36     Part    53.296318
       Full    76.844901
37     Part    51.906614
       Full    68.562115